In [2]:
import tecton, pandas as pd
from tecton import *
from tecton.types import *
from datetime import datetime, timedelta
from pprint import pprint

tecton.login('explore.tecton.ai')

tecton.set_validation_mode('auto')
tecton.conf.set('TECTON_OFFLINE_RETRIEVAL_COMPUTE_MODE', 'rift')
tecton.conf.set('TECTON_BATCH_COMPUTE_MODE', 'rift')

Re-authenticating. Switching from https://community.tecton.ai to https://explore.tecton.ai
Please visit the following link to login and access the authentication code:
https://login.tecton.ai/oauth2/default/v1/authorize?response_type=code&client_id=0oazt04n72Fkm3APE357&redirect_uri=https%3A%2F%2Fwww.tecton.ai%2Fauthorization-callback&state=1945241034329908641&scope=openid+offline_access+profile+email&code_challenge_method=S256&code_challenge=loqNygNZuRDZImbHO0gLptEcd9i_5lwtJ9paDGXiKBo
✅ Authentication successful!


In [5]:
user = Entity(name='user', join_keys=['user_id'])
restaurant = Entity(name='restaurant', join_keys=['restaurant_id'])

In [3]:
def users_post_processor(df):
    df['date_of_birth'] = df['date_of_birth'].astype(str)
    return df

users = BatchSource(
    name='users',
    batch_config=FileConfig(
        uri='s3://tecton.ai.public/tutorials/llm_demo/users.parquet',
        file_format='parquet',
        timestamp_field='timestamp',
        post_processor=users_post_processor
    ),
)

def restaurants_post_processor(df):
    import pandas
    
    df['timestamp'] = pandas.Timestamp('2020-01-01')
    df = df.rename(columns={'alias': 'restaurant_id'}).drop('id', axis=1)
    df_filtered = df[df['restaurant_id'].apply(lambda x: x.isascii())]
    return df_filtered

restaurants = BatchSource(
    name='restaurants',
    batch_config=FileConfig(
        uri='s3://tecton.ai.public/tutorials/llm_demo/yelp_dataset.parquet',
        file_format='parquet',
        timestamp_field='timestamp',
        post_processor=restaurants_post_processor
    ),
)

def ratings_post_processor(df):
    df_filtered = df[df['restaurant_id'].apply(lambda x: x.isascii())]
    return df_filtered

ratings = BatchSource(
    name='ratings',
    batch_config=FileConfig(
        uri='s3://tecton.ai.public/tutorials/llm_demo/ratings.parquet',
        file_format='parquet',
        timestamp_field='timestamp',
        post_processor=ratings_post_processor
    ),
)

users_df = users.get_dataframe().to_pandas()
restaurants_df = restaurants.get_dataframe().to_pandas()
ratings_df = ratings.get_dataframe().to_pandas()

display(users_df)
display(restaurants_df)
display(ratings_df)

BatchSource 'users': Successfully validated.
BatchSource 'restaurants': Successfully validated.
BatchSource 'ratings': Successfully validated.


,user_id,signup_time,date_of_birth,first_name,last_name,address
0,806d474e-3a3f-4f1a-8496-9739b18c8732,2022-10-03 08:05:44.974413,1968-09-29,Virginia,Ward,"4450 Rivera Tunnel, West Sharon, KS 35709"
1,fc2b5b73-7926-4756-8eaa-4445fa6bee49,2021-04-11 15:47:06.627286,1957-09-12,Kimberly,Lozano,"18348 Donald Squares Apt. 080, Melissaport, UT..."
2,55f60691-b537-445a-8eee-139b0e39d56e,2023-10-05 20:35:29.634946,1999-08-18,Nicholas,Elliott,"3354 Navarro Lock Apt. 115, East Eric, AL 11037"
3,cef517c6-a646-490b-b4e0-bd0f4c86840f,2020-09-13 00:58:10.627330,1953-10-26,Anne,Burns,"961 Michael Freeway Apt. 725, Perezstad, NV 83090"
4,c2f21f8e-0832-4d5e-954d-31735c69fe78,2022-03-17 23:15:24.076682,1955-12-16,Julie,Pitts,"4846 Robert Well, Jeremyfurt, WY 92426"
5,5c16fef7-8373-48e2-b7b1-a3577467af2a,2021-01-12 11:48:43.325414,2002-03-03,Colin,Kent,"PSC 7459, Box 6876, APO AP 80992"
6,65e9cc11-bfdb-4ed8-a5a9-34ff8964a71c,2019-11-21 09:52:08.586968,1998-01-30,Andrew,Torres,"182 Nelson Alley, New Joseph, GA 33915"
7,5680260e-6c5d-4ad4-94be-522a50d5649a,2020-07-16 06:35:12.652123,1967-04-29,Kevin,Hernandez,"7439 Kaylee Estate Apt. 652, Johnsonside, OR 8..."
8,7efe8dcf-da48-470c-9d9a-c209501eb0ad,2022-05-03 13:07:08.924195,1996-04-21,David,Carroll,"25377 Daniel Gateway Suite 820, West Stephen, ..."
9,d818b005-8e27-499f-a8e5-7ff1f92c6f50,2023-07-30 16:44:13.711670,1995-03-01,Daniel,Jackson,"030 Patrick Port Suite 858, Port Roymouth, IL ..."


,restaurant_id,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,attributes,encid,operationHours,popularItems,timestamp
0,delarosa-downtown-san-francisco,Delarosa Downtown,https://s3-media2.fl.yelpcdn.com/bphoto/UPdC4G...,False,https://www.yelp.com/biz/delarosa-downtown-san...,1738,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.9,"{'latitude': 37.78566225574775, 'longitude': -...","[restaurant_reservation, delivery]",$$,"{'address1': '37 Yerba Buena Ln', 'address2': ...",+14158727363,(415) 872-7363,282.586012,"{'business_temp_closed': None, 'menu_url': 'ht...",vu6PlPyKptsT6oEq50qOzA,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Burrata Bruschetta, Margherita Pizza, Spicy F...",2020-01-01
1,laotable-san-francisco,LaoTable,https://s3-media4.fl.yelpcdn.com/bphoto/UK4vyT...,False,https://www.yelp.com/biz/laotable-san-francisc...,1564,"[{'alias': 'laotian', 'title': 'Laotian'}, {'a...",4.0,"{'latitude': 37.7875137329102, 'longitude': -1...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '149 2nd St', 'address2': None, '...",+14152789991,(415) 278-9991,436.008373,"{'business_temp_closed': None, 'menu_url': 'ht...",VdxJdMX53XZYNZvZlr96HA,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Papaya Salad, Lao Fried Rice, Pork Belly, Red...",2020-01-01
2,the-pawn-shop-san-francisco,The Pawn Shop,https://s3-media3.fl.yelpcdn.com/bphoto/X0jMCk...,False,https://www.yelp.com/biz/the-pawn-shop-san-fra...,715,"[{'alias': 'speakeasies', 'title': 'Speakeasie...",4.6,"{'latitude': 37.781102416684746, 'longitude': ...",[restaurant_reservation],$$,"{'address1': '993 Mission St', 'address2': '',...",+14158748041,(415) 874-8041,695.625459,"{'business_temp_closed': None, 'menu_url': Non...",DLjqwgeEd4BxZT7qJKvWUw,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Patatas Bravas, Pulpo a La Plancha, Carne a L...",2020-01-01
3,liholiho-yacht-club-san-francisco-2,Liholiho Yacht Club,https://s3-media1.fl.yelpcdn.com/bphoto/IKI-M_...,False,https://www.yelp.com/biz/liholiho-yacht-club-s...,2755,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 37.78843914681705, 'longitude': -...",[delivery],$$$,"{'address1': '871 Sutter St', 'address2': None...",,,1227.613094,"{'business_temp_closed': None, 'menu_url': Non...",KkAVX3Wb7E3lP5F_f_8pXg,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Beef Tongue, Baked Hawaii, Tuna Poke, Duck Li...",2020-01-01
4,mr-east-kitchen-san-francisco-2,Mr East Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/bjTb3L...,False,https://www.yelp.com/biz/mr-east-kitchen-san-f...,675,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 37.78039, 'longitude': -122.40375}","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '276 Fifth St', 'address2': None,...",+14156587335,(415) 658-7335,478.509630,"{'business_temp_closed': None, 'menu_url': 'ht...",C_8tBcwMD5Xy-eA8QCz0Fw,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Thai Tea French Toast, Garlic Noodles, Tom Yu...",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,rooster-and-rice-san-francisco-8,Rooster & Rice,https://s3-media1.fl.yelpcdn.com/bphoto/5wkmih...,False,https://www.yelp.com/biz/rooster-and-rice-san-...,425,"[{'alias': 'thai', 'title': 'Thai'}]",4.1,"{'latitude': 37.78923, 'longitude': -122.40404}","[pickup, delivery]",$$,"{'address1': '125 Kearny St', 'address2': '', ...",+14157051500,(415) 705-1500,587.224677,"{'business_temp_closed': None, 'menu_url': '',...",UqTdHGPBx730m5D51EOI6Q,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Hainan Chicken, Dark Meat "" Paleo, Khao Mun G...",2020-01-01
288,capos-by-tony-gemignani-san-francisco-2,Capo's by Tony Gemignani,https://s3-media4.fl.yelpcdn.com/bphoto/YfQdH1...,False,https://www.yelp.com/biz/capos-by-tony-gemigna...,981,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.2,"{'latitude': 37.7985156, 'longitude': -122.4

,user_id,restaurant_id,timestamp,rating
0,65e9cc11-bfdb-4ed8-a5a9-34ff8964a71c,epic-steak-san-francisco,2023-01-05 00:07:32,4
1,cef517c6-a646-490b-b4e0-bd0f4c86840f,freshroll-vietnamese-rolls-and-bowls-san-franc...,2022-05-30 16:28:00,4
2,7efe8dcf-da48-470c-9d9a-c209501eb0ad,barcha-san-francisco,2020-07-10 07:12:50,4
3,55f60691-b537-445a-8eee-139b0e39d56e,cotogna-san-francisco,2022-06-06 00:24:33,3
4,55f60691-b537-445a-8eee-139b0e39d56e,gai-chicken-and-rice-san-francisco-3,2022-03-16 15:21:34,5
...,...,...,...,...
4867,65e9cc11-bfdb-4ed8-a5a9-34ff8964a71c,the-italian-homemade-company-san-francisco-6,2023-04-05 06:47:15,5
4868,55f60691-b537-445a-8eee-139b0e39d56e,local-kitchen-and-wine-merchant-san-francisco-2,2021-09-23 09:07:00,3
4869,d818b005-8e27-499f-a8e5-7ff1f92c6f50,hawthorn-san-francisco-3,2021-04-29 10:44:54,5
4870,5680260e-6c5d-4ad4-94be-522a50d5649a,prospect-san-francisco,2023-01-24 08:48:37,4


In [6]:
@batch_feature_view(
    description="User dimension features",
    sources=[users],
    entities=[user],
    mode="pandas",
    batch_schedule=timedelta(days=1),
    schema=[
        Field("user_id", String),
        Field("signup_time", Timestamp),
        Field("preferred_name", String),
        Field("birth_year", String),
    ],
    timestamp_field="signup_time",
    online=True,
    offline=True,
    feature_start_time=datetime(2020, 1, 1),
)
def user_features(users):
    df = users[["user_id", "signup_time"]]
    df["preferred_name"] = users["first_name"]
    df["birth_year"] = users["date_of_birth"].str.slice(0, 4)
    return df


start = datetime(2020,1,1)
end = datetime(2024,6,1)

user_features_df = user_features.get_features_in_range(start, end).to_pandas()
display(user_features_df)

BatchFeatureView 'user_features': Validating 2 of 3 dependencies. (1 already validated)
    Entity 'user': Successfully validated.
    Transformation 'user_features': Successfully validated.
BatchFeatureView 'user_features': Successfully validated.


,user_id,date_of_birth,first_name,last_name,address,_valid_to,_valid_from
0,d818b005-8e27-499f-a8e5-7ff1f92c6f50,1995-03-01,Daniel,Jackson,"030 Patrick Port Suite 858, Port Roymouth, IL ...",2024-06-01,2023-07-31
1,c2f21f8e-0832-4d5e-954d-31735c69fe78,1955-12-16,Julie,Pitts,"4846 Robert Well, Jeremyfurt, WY 92426",2024-06-01,2022-03-18
2,cef517c6-a646-490b-b4e0-bd0f4c86840f,1953-10-26,Anne,Burns,"961 Michael Freeway Apt. 725, Perezstad, NV 83090",2024-06-01,2020-09-14
3,953c6db5-89ea-4189-b286-c662591487c8,1998-07-03,Matt,Bleifer,"789 Haight St, San Francisco, CA 94117",2024-06-01,2024-01-25
4,806d474e-3a3f-4f1a-8496-9739b18c8732,1968-09-29,Virginia,Ward,"4450 Rivera Tunnel, West Sharon, KS 35709",2024-06-01,2022-10-04
5,fc2b5b73-7926-4756-8eaa-4445fa6bee49,1957-09-12,Kimberly,Lozano,"18348 Donald Squares Apt. 080, Melissaport, UT...",2024-06-01,2021-04-12
6,5c16fef7-8373-48e2-b7b1-a3577467af2a,2002-03-03,Colin,Kent,"PSC 7459, Box 6876, APO AP 80992",2024-06-01,2021-01-13
7,5680260e-6c5d-4ad4-94be-522a50d5649a,1967-04-29,Kevin,Hernandez,"7439 Kaylee Estate Apt. 652, Johnsonside, OR 8...",2024-06-01,2020-07-17
8,65e9cc11-bfdb-4ed8-a5a9-34ff8964a71c,1998-01-30,Andrew,Torres,"182 Nelson Alley, New Joseph, GA 33915",2024-06-01,2020-01-01
9,7efe8dcf-da48-470c-9d9a-c209501eb0ad,1996-04-21,David,Carroll,"25377 Daniel Gateway Suite 820, West Stephen, ...",2024-06-01,2022-05-04


In [8]:
restaurant_features_schema=[
    Field('restaurant_id', String),
    Field('timestamp', Timestamp),
    Field('name', String),
    Field('image_url', String),
    Field('is_closed', Bool),
    Field('url', String),
    Field('review_count', Int64),
    Field('categories', Array(Struct([
        Field('alias', String),
        Field('title', String)
    ]))),
    Field('rating', Float64),
    Field('coordinates', Struct([
        Field('latitude', Float64),
        Field('longitude', Float64)
    ])),
    Field('transactions', Array(String)),
    Field('price', String),
    Field('location', Struct([
        Field('address1', String),
        Field('address2', String),
        Field('address3', String),
        Field('city', String),
        Field('zip_code', String),
        Field('country', String),
        Field('state', String),
        Field('display_address', Array(String))
    ])),
    Field('phone', String),
    Field('display_phone', String),
    Field('distance', Float64),
    Field('attributes', Struct([
        Field('business_temp_closed', Bool),
        Field('menu_url', String),
        Field('open24_hours', Bool),
        Field('waitlist_reservation', Bool)
    ])),
    Field('encid', String),
    Field('operationHours', Struct([
        Field('regularHoursMergedWithSpecialHoursForCurrentWeek', Array(Struct([
            Field('dayOfWeekShort', String),
            Field('regularHoursRaw', Array(Array(Int64))),
            Field('__typename', String)
        ]))),
        Field('__typename', String)
    ])),
    Field('popularItems', Array(String))
]

@batch_feature_view(
    description='Restaurant dimension features',
    sources=[restaurants],
    entities=[restaurant],
    mode='pandas',
    batch_schedule=timedelta(days=1),
    schema=restaurant_features_schema,
    timestamp_field='timestamp',
    online=True,
    offline=True,
    feature_start_time=datetime(2020,1,1),
    environment='tecton-rift-core-0.9.0'
)
def restaurant_features(restaurants):
    return restaurants[['restaurant_id', 'timestamp', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance', 'attributes', 'encid', 'operationHours', 'popularItems']]
start = datetime(2020,1,1)
end = datetime(2024,6,1)

restaurant_features_df = restaurant_features.get_features_in_range(start, end).to_pandas()
display(restaurant_features_df)

BatchFeatureView 'restaurant_features': Validating 1 of 3 dependencies. (2 already validated)
    Transformation 'restaurant_features': Successfully validated.
BatchFeatureView 'restaurant_features': Successfully validated.


,restaurant_id,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,...,location,phone,display_phone,distance,attributes,encid,operationHours,popularItems,_valid_to,_valid_from
0,corzetti-san-francisco-3,Corzetti,https://s3-media1.fl.yelpcdn.com/bphoto/Ck_EHG...,False,https://www.yelp.com/biz/corzetti-san-francisc...,153,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.2,"{'latitude': 37.7875656270973, 'longitude': -1...",[],...,"{'address1': '398 Geary St', 'address2': '', '...",+14153145288,(415) 314-5288,814.888137,"{'business_temp_closed': None, 'menu_url': 'ht...",nQ9h1szCd0JP1hKjagi2cQ,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Fazzoletti Al Pesto, Fritto Misto, Panna Cott...",2024-06-01,2020-01-02
1,mister-jius-san-francisco,Mister Jiu's,https://s3-media3.fl.yelpcdn.com/bphoto/k6UE-P...,False,https://www.yelp.com/biz/mister-jius-san-franc...,1037,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.7,"{'latitude': 37.7937724, 'longitude': -122.406...",[delivery],...,"{'address1': '28 Waverly Pl', 'address2': '', ...",+14158579688,(415) 857-9688,1148.178580,"{'business_temp_closed': None, 'menu_url': Non...",p22TPx1EFZcuEzl2qhuzGA,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Sourdough Scallion Pancake, Cheong Fun, Heart...",2024-06-01,2020-01-02
2,tycoon-kitchen-san-francisco-2,Tycoon Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/67JsO8...,False,https://www.yelp.com/biz/tycoon-kitchen-san-fr...,81,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.7,"{'latitude': 37.78859, 'longitude': -122.39776}",[],...,"{'address1': '47 Minna St', 'address2': None, ...",+14154296665,(415) 429-6665,565.435747,"{'business_temp_closed': None, 'menu_url': 'ht...",LqRy_zV0esyiYNp6sp8J2Q,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Drunken Noodles, Pad See Ew, Spring Rolls, Th...",2024-06-01,2020-01-02
3,laotable-san-francisco,LaoTable,https://s3-media4.fl.yelpcdn.com/bphoto/UK4vyT...,False,https://www.yelp.com/biz/laotable-san-francisc...,1564,"[{'alias': 'laotian', 'title': 'Laotian'}, {'a...",4.0,"{'latitude': 37.7875137329102, 'longitude': -1...","[restaurant_reservation, pickup, delivery]",...,"{'address1': '149 2nd St', 'address2': None, '...",+14152789991,(415) 278-9991,436.008373,"{'business_temp_closed': None, 'menu_url': 'ht...",VdxJdMX53XZYNZvZlr96HA,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Papaya Salad, Lao Fried Rice, Pork Belly, Red...",2024-06-01,2020-01-02
4,alexanders-steakhouse-san-francisco-7,Alexander's Steakhouse,https://s3-media1.fl.yelpcdn.com/bphoto/-v5k5f...,False,https://www.yelp.com/biz/alexanders-steakhouse...,1938,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.1,"{'latitude': 37.77931, 'longitude': -122.39569}","[pickup, delivery]",...,"{'address1': '448 Brannan St', 'address2': '',...",+14154951111,(415) 495-1111,758.277313,"{'business_temp_closed': None, 'menu_url': 'ht...",PLx_VnZyFmxTS1zNTaDJxA,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Prime Rib, Cab Filet Mignon, Bone-in New York...",2024-06-01,2020-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,lukes-lobster-soma-san-francisco-4,Luke's Lobster SoMa,https://s3-media2.fl.yelpcdn.com/bphoto/9if7kc...,False,https://www.yelp.com/biz/lukes-lobster-soma-sa...,560,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.8,"{'latitude': 37.78802274999075, 'longitude': -...",[delivery],...,"{'address1': '92 2nd St', 'address2': '', 'add...",+14156587443,(415) 658-7443,434.914614,"{'business_temp_closed': None, 'menu_url': 'ht...",XWGSjN82azh6Op-tG4659Q,{'regularHoursMergedWithSpecialHoursForCurrent...,"[Lobster Roll, Crab Roll, Clam Chowder, Shrimp...",2024-06-01,2020-01-02
288,zzan-san-francisco-2,ZZAN,https://s3-media2.fl.yelpcdn.com/bphoto/en355f...,False,https://www.yelp.com/biz/zzan-san-francisco-2?...,714,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.3,"{'latitude': 37.787539, 'longitude': -122.412386}","[delivery, pick

In [12]:

@batch_feature_view(
    description='User restaurant engagement features',
    sources=[ratings],
    entities=[user],
    mode='pandas',
    batch_schedule=timedelta(days=1),
    aggregation_interval=timedelta(days=1),
    schema=[Field('user_id', String), Field('restaurant_id', String), Field('timestamp', Timestamp), Field('rating', Int64)],
    timestamp_field='timestamp',
    environment='tecton-rift-core-0.9.0',
    aggregation_secondary_key='restaurant_id',
    aggregations=[
        Aggregation(name='users_average_rating_by_restaurant_last_5y', column='rating', function='mean', time_window=timedelta(days=365*5)),
        Aggregation(name='users_total_visits_by_restaurant_last_5y', column='rating', function='count', time_window=timedelta(days=365*5))
    ],
    online=True,
    offline=True,
    feature_start_time=datetime(2020,1,1)
)
def user_engagement_by_restaurant_features(ratings):
    return ratings[['user_id', 'restaurant_id', 'timestamp', 'rating']]




start = datetime(2020,1,1)
end = datetime(2024,6,1)

user_engagement_by_restaurant_features_df = user_engagement_by_restaurant_features.get_features_in_range(start, end).to_pandas()
display(user_engagement_by_restaurant_features_df)

BatchFeatureView 'user_engagement_by_restaurant_features': Validating 1 of 3 dependencies. (2 already validated)
    Transformation 'user_engagement_by_restaurant_features': Successfully validated.
BatchFeatureView 'user_engagement_by_restaurant_features': Successfully validated.


,user_id,restaurant_id_keys_1825d,users_average_rating_by_restaurant_last_5y,users_total_visits_by_restaurant_last_5y,_valid_to,_valid_from
0,55f60691-b537-445a-8eee-139b0e39d56e,"[54-mint-ristorante-italiano-san-francisco-2, ...","[4.0, 3.5, 4.5, 4.0, 5.0, 4.0, 4.0, 4.0, 3.0, ...","[2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 2, ...",2022-03-21,2022-03-20
1,5680260e-6c5d-4ad4-94be-522a50d5649a,"[54-mint-ristorante-italiano-san-francisco-2, ...","[3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, ...",2021-07-10,2021-06-24
2,5680260e-6c5d-4ad4-94be-522a50d5649a,"[54-mint-ristorante-italiano-san-francisco-2, ...","[3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, ...",2021-08-08,2021-08-02
3,5680260e-6c5d-4ad4-94be-522a50d5649a,"[54-mint-ristorante-italiano-san-francisco-2, ...","[3.0, 4.0, 4.0, 3.5, 4.0, 3.5, 4.0, 5.0, 4.0, ...","[1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2023-02-02,2023-01-29
4,5680260e-6c5d-4ad4-94be-522a50d5649a,"[54-mint-ristorante-italiano-san-francisco-2, ...","[3.0, 4.0, 4.0, 3.5, 4.0, 3.5, 4.0, 5.0, 4.0, ...","[1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 3, ...",2023-09-06,2023-09-03
...,...,...,...,...,...,...
4312,d818b005-8e27-499f-a8e5-7ff1f92c6f50,"[707-sutter-san-francisco, abrazo-san-francisc...","[4.0, 4.0, 5.0, 3.5, 4.0, 3.5, 4.0, 4.0, 4.0, ...","[1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, ...",2021-04-30,2021-04-24
4313,d818b005-8e27-499f-a8e5-7ff1f92c6f50,"[54-mint-ristorante-italiano-san-francisco-2, ...","[5.0, 4.0, 4.0, 5.0, 4.0, 4.0, 3.5, 4.5, 4.0, ...","[1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, ...",2022-11-26,2022-11-23
4314,d818b005-8e27-499f-a8e5-7ff1f92c6f50,"[54-mint-ristorante-italiano-san-francisco-2, ...","[5.0, 4.0, 4.0, 5.0, 4.0, 4.0, 3.5, 3.0, 4.5, ...","[1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 2, 1, 1, ...",2023-09-10,2023-09-06
4315,d818b005-8e27-499f-a8e5-7ff1f92c6f50,"[54-mint-ristorante-italiano-san-francisco-2, ...","[5.0, 4.0, 4.0, 5.0, 4.0, 4.0, 3.5, 3.0, 4.5, ...","[1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 3, 1, 2, 1, 1, ...",2023-10-27,2023-10-26


In [19]:
@on_demand_feature_view(
    description='User ratings and total visits by restaurant',
    sources=[user_engagement_by_restaurant_features],
    mode='python',
    schema=[Field('user_ratings_and_total_visits_by_restaurant', Array(Struct([Field('restaurant_id', String), Field('average_rating', Float64), Field('total_visits', Int64)])))],
    tags={'llm_enabled': 'True'}
)
def user_ratings_and_total_visits_by_restaurant(user_ratings_and_total_visits_by_restaurant_raw):
    restaurants = user_ratings_and_total_visits_by_restaurant_raw['restaurant_id_keys_1825d']
    ratings = user_ratings_and_total_visits_by_restaurant_raw['users_average_rating_by_restaurant_last_5y']
    visits = user_ratings_and_total_visits_by_restaurant_raw['users_total_visits_by_restaurant_last_5y']
    
    result = []
    for restaurant, rating, visit in zip(restaurants, ratings, visits):
        result.append({
            'restaurant_id': restaurant,
            'average_rating': rating,
            'total_visits': visit
        })
    
    return {'user_ratings_and_total_visits_by_restaurant': result}

user_ratings_and_total_visits_by_restaurant.get_features_for_events(events=user_engagement_by_restaurant_features_df, timestamp_key='_valid_to').to_pandas()[['user_ratings_and_total_visits_by_restaurant__user_ratings_and_total_visits_by_restaurant']].iloc[1,0]

OnDemandFeatureView 'user_ratings_and_total_visits_by_restaurant': Validating 1 of 2 dependencies. (1 already validated)
    Transformation 'user_ratings_and_total_visits_by_restaurant': Successfully validated.
OnDemandFeatureView 'user_ratings_and_total_visits_by_restaurant': Successfully validated.


/opt/anaconda3/envs/pycon-demo/lib/python3.8/site-packages/tecton_core/query/pandas/nodes.py:381: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  odfv_inputs[input_name] = odfv_inputs[input_name].to_dict("records")


[{'restaurant_id': '54-mint-ristorante-italiano-san-francisco-2',
  'average_rating': 3.0,
  'total_visits': 1},
 {'restaurant_id': 'abrazo-san-francisco-2',
  'average_rating': 4.0,
  'total_visits': 1},
 {'restaurant_id': 'ajisen-ramen-san-francisco',
  'average_rating': 4.0,
  'total_visits': 1},
 {'restaurant_id': 'alchemist-bar-and-lounge-san-francisco',
  'average_rating': 3.0,
  'total_visits': 1},
 {'restaurant_id': 'angler-san-francisco',
  'average_rating': 4.0,
  'total_visits': 1},
 {'restaurant_id': 'basil-thai-restaurant-and-bar-san-francisco',
  'average_rating': 4.0,
  'total_visits': 1},
 {'restaurant_id': 'bin-480-san-francisco',
  'average_rating': 3.0,
  'total_visits': 1},
 {'restaurant_id': 'bocado-san-francisco',
  'average_rating': 4.0,
  'total_visits': 1},
 {'restaurant_id': 'bodega-san-francisco-3',
  'average_rating': 4.0,
  'total_visits': 1},
 {'restaurant_id': 'bota-tapas-and-paella-bar-san-francisco-3',
  'average_rating': 4.0,
  'total_visits': 1},
 {'r

In [10]:
from tecton.aggregation_functions import last

@batch_feature_view(
    description='Restaurant features',
    sources=[ratings],
    entities=[user],
    mode='pandas',
    batch_schedule=timedelta(days=1),
    aggregation_interval=timedelta(days=1),
    schema=[Field('user_id', String), Field('restaurant_id', String), Field('timestamp', Timestamp), Field('rating', Int64)],
    timestamp_field='timestamp',
    environment='tecton-rift-core-0.9.0',
    aggregations=[
        Aggregation(name='last_100_restaurants_eaten_at', column='restaurant_id', function=last(100), time_window=timedelta(days=365*5)),
        Aggregation(name='last_100_restaurant_ratings_for_last_100_restaurants_eaten_at', column='rating', function=last(100), time_window=timedelta(days=365*5)),
    ]
)
def user_last_restaurant_ratings(ratings):
    return ratings[['user_id', 'timestamp', 'restaurant_id', 'rating']]

start = datetime(2020,1,1)
end = datetime(2024,6,1)

user_last_restaurant_ratings_df = user_last_restaurant_ratings.get_features_in_range(start, end).to_pandas()
display(user_last_restaurant_ratings_df)

BatchFeatureView 'user_last_restaurant_ratings': Validating 1 of 3 dependencies. (2 already validated)
    Transformation 'user_last_restaurant_ratings': Successfully validated.
BatchFeatureView 'user_last_restaurant_ratings': Successfully validated.


,user_id,last_100_restaurants_eaten_at,last_100_restaurant_ratings_for_last_100_restaurants_eaten_at,_valid_to,_valid_from
0,55f60691-b537-445a-8eee-139b0e39d56e,"[prospect-san-francisco, absinthe-brasserie-an...","[5, 4]",2024-06-01,2024-05-28
1,5680260e-6c5d-4ad4-94be-522a50d5649a,[harborview-restaurant-and-bar-san-francisco-2...,"[3, 4, 5, 4]",2022-10-13,2022-10-02
2,5680260e-6c5d-4ad4-94be-522a50d5649a,"[gusto-pinsa-romana-san-francisco-4, village-t...","[4, 4]",2023-08-01,2023-07-29
3,5680260e-6c5d-4ad4-94be-522a50d5649a,"[fermentation-lab-san-francisco-3, bouche-san-...","[3, 4, 4]",2024-06-01,2024-05-21
4,5c16fef7-8373-48e2-b7b1-a3577467af2a,"[betty-lous-seafood-and-grill-san-francisco, k...","[4, 4, 4, 4, 3, 5, 4, 4]",2020-04-02,2020-04-01
...,...,...,...,...,...
4312,d818b005-8e27-499f-a8e5-7ff1f92c6f50,"[tommaso-ristorante-italiano-san-francisco-2, ...","[5, 5, 5, 4, 3]",2021-09-27,2021-09-26
4313,d818b005-8e27-499f-a8e5-7ff1f92c6f50,[freshroll-vietnamese-rolls-and-bowls-san-fran...,"[4, 4, 3, 4, 4, 4, 4, 3, 4]",2022-03-07,2022-02-23
4314,fc2b5b73-7926-4756-8eaa-4445fa6bee49,"[the-barrel-room-san-francisco-3, lers-ros-san...","[4, 4, 5, 4, 4, 4, 5, 4, 4, 4, 4, 5, 5, 4, 4, ...",2021-05-17,2021-05-14
4315,fc2b5b73-7926-4756-8eaa-4445fa6bee49,"[alexanders-steakhouse-san-francisco-7, boulev...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 5, 4, 5, ...",2023-10-24,2023-10-14


In [11]:
@on_demand_feature_view(
    sources=[user_last_restaurant_ratings],
    mode='python',
    schema=[Field('user_last_100_restaurants_eaten_at_with_ratings', Array(Struct([Field('restaurant_id', String), Field('rating', Int64)])))],
 )
def user_last_100_restaurants_eaten_at_with_ratings(user_last_restaurant_ratings):
    restaurants = user_last_restaurant_ratings['last_100_restaurants_eaten_at']
    ratings = user_last_restaurant_ratings['last_100_restaurant_ratings_for_last_100_restaurants_eaten_at']
    restaurants_ratings = [{"restaurant_id": restaurant, "rating": rating} for restaurant, rating in zip(restaurants, ratings)]
    return {'user_last_100_restaurants_eaten_at_with_ratings': restaurants_ratings}

user_last_100_restaurants_eaten_at_with_ratings.get_features_for_events(events=user_last_restaurant_ratings_df, timestamp_key='_valid_to').to_pandas()[['user_last_100_restaurants_eaten_at_with_ratings__user_last_100_restaurants_eaten_at_with_ratings']].iloc[1,0]

OnDemandFeatureView 'user_last_100_restaurants_eaten_at_with_ratings': Validating 1 of 2 dependencies. (1 already validated)
    Transformation 'user_last_100_restaurants_eaten_at_with_ratings': Successfully validated.
OnDemandFeatureView 'user_last_100_restaurants_eaten_at_with_ratings': Successfully validated.


/opt/anaconda3/envs/pycon-demo/lib/python3.8/site-packages/tecton_core/query/pandas/nodes.py:381: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  odfv_inputs[input_name] = odfv_inputs[input_name].to_dict("records")


[{'restaurant_id': 'aphotic-san-francisco', 'rating': 4},
 {'restaurant_id': 'pacific-cocktail-haven-san-francisco-2', 'rating': 3},
 {'restaurant_id': 'shelbys-rooftop-lounge-san-francisco', 'rating': 4}]

In [13]:
@batch_feature_view(
    description='User ratings and total visits by cuisine',
    sources=[ratings, restaurants],
    entities=[user],
    mode='pandas',
    batch_schedule=timedelta(days=1),
    aggregation_interval=timedelta(days=1),
    schema=[Field('user_id', String), Field('cuisine', String), Field('timestamp', Timestamp), Field('rating', Int64)],
    timestamp_field='timestamp',
    environment='tecton-rift-core-0.9.0',
    aggregation_secondary_key='cuisine',
    aggregations=[
        Aggregation(name='users_average_rating_by_cuisine_last_5y', column='rating', function='mean', time_window=timedelta(days=365*5)),
        Aggregation(name='users_total_visits_by_cuisine_last_5y', column='rating', function='count', time_window=timedelta(days=365*5))
    ],
    online=True,
    offline=True,
    feature_start_time=datetime(2020,1,1)
)
def user_ratings_and_total_visits_by_cuisine_raw(ratings, restaurants):
    df = ratings.merge(restaurants, on='restaurant_id', how='left').explode('categories')
    df['cuisine'] = df['categories'].apply(lambda x: x['title'])
    df['rating'] = df['rating_x']
    df['timestamp'] = df['timestamp_x']
    return df[['user_id', 'cuisine', 'timestamp', 'rating']]


start = datetime(2020,1,1)
end = datetime(2024,6,1)

user_engagement_by_cuisine_features_df = user_ratings_and_total_visits_by_cuisine_raw.run_transformation(start, end).to_pandas()
display(user_engagement_by_cuisine_features_df)

BatchFeatureView 'user_ratings_and_total_visits_by_cuisine_raw': Validating 1 of 4 dependencies. (3 already validated)
    Transformation 'user_ratings_and_total_visits_by_cuisine_raw': Successfully validated.
BatchFeatureView 'user_ratings_and_total_visits_by_cuisine_raw': Successfully validated.


,user_id,cuisine,rating,timestamp
0,65e9cc11-bfdb-4ed8-a5a9-34ff8964a71c,Steakhouses,4,2023-01-05 00:07:32
1,65e9cc11-bfdb-4ed8-a5a9-34ff8964a71c,Bars,4,2023-01-05 00:07:32
2,65e9cc11-bfdb-4ed8-a5a9-34ff8964a71c,Venues & Event Spaces,4,2023-01-05 00:07:32
3,cef517c6-a646-490b-b4e0-bd0f4c86840f,Vietnamese,4,2022-05-30 16:28:00
4,cef517c6-a646-490b-b4e0-bd0f4c86840f,Sandwiches,4,2022-05-30 16:28:00
...,...,...,...,...
11627,d818b005-8e27-499f-a8e5-7ff1f92c6f50,Venues & Event Spaces,5,2021-04-29 10:44:54
11628,5680260e-6c5d-4ad4-94be-522a50d5649a,New American,4,2023-01-24 08:48:37
11629,fc2b5b73-7926-4756-8eaa-4445fa6bee49,Thai,4,2021-02-13 20:58:40
11630,fc2b5b73-7926-4756-8eaa-4445fa6bee49,Breakfast & Brunch,4,2021-02-13 20:58:40


In [17]:
@on_demand_feature_view(
    description='User ratings and total visits by cuisine',
    sources=[user_ratings_and_total_visits_by_cuisine_raw],
    mode='python',
    schema=[Field('user_ratings_and_total_visits_by_cuisine', Array(Struct([Field('cuisine', String), Field('average_rating', Float64), Field('total_visits', Int64)])))],
    tags={'llm_enabled': 'True'}
)
def user_ratings_and_total_visits_by_cuisine(user_ratings_and_total_visits_by_cuisine_raw):
    cuisines = user_ratings_and_total_visits_by_cuisine_raw['cuisine_keys_1825d']
    ratings = user_ratings_and_total_visits_by_cuisine_raw['users_average_rating_by_cuisine_last_5y']
    visits = user_ratings_and_total_visits_by_cuisine_raw['users_total_visits_by_cuisine_last_5y']
    
    result = []
    for cuisine, rating, visit in zip(cuisines, ratings, visits):
        result.append({
            'cuisine': cuisine,
            'average_rating': rating,
            'total_visits': visit
        })
    
    return {'user_ratings_and_total_visits_by_cuisine': result}

user_ratings_and_total_visits_by_cuisine.get_features_for_events(events=user_last_restaurant_ratings_df, timestamp_key='_valid_to').to_pandas() [['user_ratings_and_total_visits_by_cuisine__user_ratings_and_total_visits_by_cuisine']].iloc[1,0]

OnDemandFeatureView 'user_ratings_and_total_visits_by_cuisine': Validating 1 of 2 dependencies. (1 already validated)
    Transformation 'user_ratings_and_total_visits_by_cuisine': Successfully validated.
OnDemandFeatureView 'user_ratings_and_total_visits_by_cuisine': Successfully validated.


[{'cuisine': 'Afghan', 'average_rating': 4.0, 'total_visits': 1},
 {'cuisine': 'American', 'average_rating': 4.0, 'total_visits': 21},
 {'cuisine': 'Arcades', 'average_rating': 4.0, 'total_visits': 3},
 {'cuisine': 'Asian Fusion', 'average_rating': 4.3, 'total_visits': 10},
 {'cuisine': 'Bakeries', 'average_rating': 4.0, 'total_visits': 1},
 {'cuisine': 'Barbeque',
  'average_rating': 4.333333333333333,
  'total_visits': 6},
 {'cuisine': 'Bars', 'average_rating': 4.041666666666667, 'total_visits': 24},
 {'cuisine': 'Beer Bar', 'average_rating': 3.5, 'total_visits': 2},
 {'cuisine': 'Beer, Wine & Spirits', 'average_rating': 4.0, 'total_visits': 2},
 {'cuisine': 'Breakfast & Brunch',
  'average_rating': 3.9130434782608696,
  'total_visits': 23},
 {'cuisine': 'British', 'average_rating': 4.0, 'total_visits': 1},
 {'cuisine': 'Burgers',
  'average_rating': 4.142857142857143,
  'total_visits': 7},
 {'cuisine': 'Cafes', 'average_rating': 3.5, 'total_visits': 2},
 {'cuisine': 'Cajun/Creole', 